# Exercício: Modelo de linguagem (Bengio 2003) - MLP + Embeddings

# Parâmetros

In [1]:
# Livros (testando com dom casmurro, memórias póstumas e quincas borda, pra dar pelo menos 20.000 palavras)
#urls = ["https://www.gutenberg.org/cache/epub/55752/pg55752.txt", "https://www.gutenberg.org/cache/epub/54829/pg54829.txt", "https://www.gutenberg.org/cache/epub/55682/pg55682.txt"]

# Livros (O Guarani)
urls = ["https://www.gutenberg.org/ebooks/67724.txt.utf-8", "https://www.gutenberg.org/ebooks/67725.txt.utf-8"]

# Dados do vocabulário
UNK = "<unk>"
vocab_size_desejado_sem_UNK = 3000 # Não considera o UNK
vocab_size = vocab_size_desejado_sem_UNK + 1

# Dados de treinamento
context_size = 9 # número de palavras de entrada. O target é a próxima palavra
num_epochs = 50 # usado pra fazer overfit no modelo e ajudar na verificação do treinamento
test_size = 0.2
seed = 18
batch_size=128
m = 64 # tamanho dos embeddings
h = 200 # tamanho da camada oculta
lr = 0.03
momentum = 0.9

# Tratamentos dos dados

## Download e agrupamento em parágrafos

In [2]:
import requests

def paragrafo_valido(paragrafo):
  # Remove:
  # - parágrafos curtos demais
  # - parágrafos do índice (que tem a string ....)
  return len(paragrafo) > 10 and '....' not in paragrafo

def carregar_paragrafos_livro(url, n_linhas_para_print=20):
  # Baixar o arquivo de texto
  response = requests.get(url)
  texto = response.text

  # Encontrar o início e o fim do conteúdo principal do livro
  inicio = texto.find("*** START OF THE PROJECT GUTENBERG EBOOK")
  fim = texto.find("*** END OF THE PROJECT GUTENBERG EBOOK")

  # Extrair o conteúdo principal do livro
  conteudo = texto[inicio:fim].replace('\r','')

  # Dividir o conteúdo em parágrafos e processar o conteúdo
  paragrafos = []

  # Cada parágrafo é separado por dois \n
  # Dentro de cada parágrafo, junta as linhas (remove) e faz um trim
  # Apenas considera os parágrafos que tem pelo menos 10 caracteres
  for paragrafo in conteudo.split("\n\n"):
    paragrafo = paragrafo.replace('\n', ' ').strip()
    if paragrafo_valido(paragrafo):
      paragrafos.append(paragrafo)

  for p in paragrafos[0:n_linhas_para_print]:
    print(p)

  return paragrafos

paragrafos = []
for i, url in enumerate(urls, 1):
  print(f'-------------- Livro {i} ---------------')
  paragrafos.extend(carregar_paragrafos_livro(url))

print('-------------- -------------------------')
print(f'Total de parágrafos: {len(paragrafos)}')

-------------- Livro 1 ---------------
*** START OF THE PROJECT GUTENBERG EBOOK O GUARANY: ROMANCE BRAZILEIRO, VOL. 1 (OF 2) ***
J. DE ALENCAR
ROMANCE BRAZILEIRO
QUINTA EDIÇÃO
TOMO PRIMEIRO
RIO DE JANEIRO
B.-L. GARNIER, LIVREIRO-EDITOR
71, RUA DO OUVIDOR, 71
PARIS.--E. MELLIER, 17, RUA SÉGUIER.
Ficão reservados os direitos de propriedade.
Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
Esta nova edição devia dar satisfação do empenho, que a extrema benevolencia do publico ledor, tão minguado ainda, mudou em bem para divida de reconhecimento.
Mais do que podia fiou de si o autor. Relendo a obra depois de annos, achou elle tão mau e incorrecto quando escrevera, que para bem corrigir, fora mister escrever de novo. Para tanto lhe carece o tempo e sobra o tedio de um labor ingrato.
Cingio-se pois ás pequenas emendas que toleravão o plano da obra e o desalinho de um estylo não castigado.
PRIMEIRA 

In [3]:
paragrafos[3]

'QUINTA EDIÇÃO'

## Tokenizador

Define um tokenizador simples. A ideia desse tokenizador é manter as palavras e os sinais de pontuação. Quero testar gerar tokens também para os sinais de pontuação.

*Gerado com ChatGPT.*

In [4]:
import re

def tokenizar(texto):
  texto = texto.lower()

  # Força os 3 pontos aparecerem juntos
  texto = texto.replace('...', 'SUBSTITUIRPORTRESPONTOS')

  # Define a expressão regular que captura palavras e sinais de pontuação
  padrao = r'\w+|[^\w\s]'

  # Usa o método findall para encontrar todas as ocorrências que se encaixam no padrão
  tokens = re.findall(padrao, texto)

  return tokens

print(tokenizar('Teste. Será que vai manter a pontuação?'))

['teste', '.', 'será', 'que', 'vai', 'manter', 'a', 'pontuação', '?']


## Geração do vocabulário

Agora vamos gerar o vocabulário.

In [5]:
%%time
from collections import Counter
counter = Counter()

def gerar_vocabulario(paragrafos, vocab_size_sem_UNK):
  for p in paragrafos:
    # Update com os tokens de cada parágrafo
    counter.update(tokenizar(p))

  # Considera apenas as palavras mais frequentes. Adiciona, na posição 0, o token UNK
  most_frequent_words = [UNK] + sorted(counter, key=counter.get, reverse=True)[:vocab_size_sem_UNK]
  # vocab é um mapa de palavras para o índice correspondente. O mapa leva a palavra para um índice entre [0, vocab_size]
  # (o tamanho é vocab_size + 1), com o índice 0 apontando para UNK
  vocab = {word: i for i, word in enumerate(most_frequent_words)}

  return len(most_frequent_words), vocab, most_frequent_words

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 16.7 µs


In [6]:
vocab_size, vocab, most_frequent_words = gerar_vocabulario(paragrafos, vocab_size_desejado_sem_UNK)

print('Tamanho do vocabulário (considera UNK): ', vocab_size)

print('Posição 0: ', most_frequent_words[0])
print('Índice do UNK: ', vocab[UNK])
print('------------')
print('Posição 200: ', most_frequent_words[200])
print(f'Índice de {most_frequent_words[200]}: ', vocab[most_frequent_words[200]])

Tamanho do vocabulário (considera UNK):  3001
Posição 0:  <unk>
Índice do UNK:  0
------------
Posição 200:  tambem
Índice de tambem:  200


## Encoder de frases

In [7]:
def encode_sentence(sentence, vocab):
  # Obs.: tem que usar o mesmo tokenizador que foi gerado o vocabulário
  return [vocab.get(word, 0) for word in tokenizar(sentence)]

In [8]:
def decode_sentence(sentence, most_frequent_words):
  words = [most_frequent_words[code] for code in sentence]
  return ' '.join(words)

In [9]:
# Teste do encode/decode
frase = "E ele pegou a árvore e arrancou do chão."

frase_encodada = encode_sentence(frase, vocab)
frase_reconstruida = decode_sentence(frase_encodada, most_frequent_words)

print('Original:')
print(frase)
print('Encodada:')
print(frase_encodada)
print('Reconstruída:')
print(frase_reconstruida)
print("--------------------------------------")

frase = "E no seminario me disseram que não."

frase_encodada = encode_sentence(frase, vocab)
frase_reconstruida = decode_sentence(frase_encodada, most_frequent_words)

print('Original:')
print(frase)
print('Encodada:')
print(frase_encodada)
print('Reconstruída:')
print(frase_reconstruida)


Original:
E ele pegou a árvore e arrancou do chão.
Encodada:
[8, 0, 0, 4, 0, 8, 1640, 12, 378, 3]
Reconstruída:
e <unk> <unk> a <unk> e arrancou do chão .
--------------------------------------
Original:
E no seminario me disseram que não.
Encodada:
[8, 25, 0, 45, 0, 5, 13, 3]
Reconstruída:
e no <unk> me <unk> que não .


# Dataset

Para cada parágrafo, é necessário gerar os dados de treinamento. Supondo que a frase é "eu gosto de pizza." e vamos usar uma janela de contexto igual a 2, a ideia é que essa frase gere o seguinte conjunto de treinamento:

input -> target

[UNK, "eu"] -> "gosto" (ESSE CASO NÃO SERÁ CONSIDERADO POR ENQUANTO)

["eu", "gosto"] -> "de"

["gosto", "de"] -> "pizza"

["de", "pizza"] -> "."

In [10]:
def gera_inputs_e_targets_para_array(array, n):
  # Faz uma janela deslizante de tamanho n no array

  janelas = []
  targets = []

  for i in range(len(array) - n):
    janela = array[i:i+n]
    janelas.append(janela)
    targets.append(array[i+n])

  return janelas, targets

# Exemplo de uso
exemplo = "eu gosto de pizza .".split()

for n in range(1, 4):
  print(f'Testando para janela de tamanho {n}')
  inputs, targets = gera_inputs_e_targets_para_array(exemplo, n)

  # Testa
  for input_target in zip(inputs, targets):
    print(f'{input_target[0]} -> {input_target[1]}')
  print('------------------------------')

Testando para janela de tamanho 1
['eu'] -> gosto
['gosto'] -> de
['de'] -> pizza
['pizza'] -> .
------------------------------
Testando para janela de tamanho 2
['eu', 'gosto'] -> de
['gosto', 'de'] -> pizza
['de', 'pizza'] -> .
------------------------------
Testando para janela de tamanho 3
['eu', 'gosto', 'de'] -> pizza
['gosto', 'de', 'pizza'] -> .
------------------------------


In [11]:
# Testa com um parágrafo real e o tamanho do contexto configurado
i = 60
inputs, targets = gera_inputs_e_targets_para_array(tokenizar(paragrafos[i]), context_size)
print(paragrafos[i])
for input_target in zip(inputs, targets):
  print(f'{input_target[0]} -> {input_target[1]}')

--Aqui sou portuguez! Aqui pode respirar á vontade um coração leal, que nunca desmentio a fé do juramento. Nesta terra que me foi dada pelo meu rei, e conquistada pelo meu braço, nesta terra livre, tu reinarás, Portugal, como viverás n'alma de teus filhos. Eu o juro!
['-', '-', 'aqui', 'sou', 'portuguez', '!', 'aqui', 'pode', 'respirar'] -> á
['-', 'aqui', 'sou', 'portuguez', '!', 'aqui', 'pode', 'respirar', 'á'] -> vontade
['aqui', 'sou', 'portuguez', '!', 'aqui', 'pode', 'respirar', 'á', 'vontade'] -> um
['sou', 'portuguez', '!', 'aqui', 'pode', 'respirar', 'á', 'vontade', 'um'] -> coração
['portuguez', '!', 'aqui', 'pode', 'respirar', 'á', 'vontade', 'um', 'coração'] -> leal
['!', 'aqui', 'pode', 'respirar', 'á', 'vontade', 'um', 'coração', 'leal'] -> ,
['aqui', 'pode', 'respirar', 'á', 'vontade', 'um', 'coração', 'leal', ','] -> que
['pode', 'respirar', 'á', 'vontade', 'um', 'coração', 'leal', ',', 'que'] -> nunca
['respirar', 'á', 'vontade', 'um', 'coração', 'leal', ',', 'que', 'n

In [12]:
# Testa com um parágrafo real, mas agora ele encodado e o tamanho do contexto configurado
inputs, targets = gera_inputs_e_targets_para_array(encode_sentence(paragrafos[i], vocab), context_size)
print(paragrafos[i])
for input_target in zip(inputs, targets):
  print(f'{input_target[0]} -> {input_target[1]}')

--Aqui sou portuguez! Aqui pode respirar á vontade um coração leal, que nunca desmentio a fé do juramento. Nesta terra que me foi dada pelo meu rei, e conquistada pelo meu braço, nesta terra livre, tu reinarás, Portugal, como viverás n'alma de teus filhos. Eu o juro!
[2, 2, 254, 779, 778, 21, 254, 646, 0] -> 32
[2, 254, 779, 778, 21, 254, 646, 0, 32] -> 351
[254, 779, 778, 21, 254, 646, 0, 32, 351] -> 11
[779, 778, 21, 254, 646, 0, 32, 351, 11] -> 133
[778, 21, 254, 646, 0, 32, 351, 11, 133] -> 1294
[21, 254, 646, 0, 32, 351, 11, 133, 1294] -> 1
[254, 646, 0, 32, 351, 11, 133, 1294, 1] -> 5
[646, 0, 32, 351, 11, 133, 1294, 1, 5] -> 246
[0, 32, 351, 11, 133, 1294, 1, 5, 246] -> 0
[32, 351, 11, 133, 1294, 1, 5, 246, 0] -> 4
[351, 11, 133, 1294, 1, 5, 246, 0, 4] -> 931
[11, 133, 1294, 1, 5, 246, 0, 4, 931] -> 12
[133, 1294, 1, 5, 246, 0, 4, 931, 12] -> 732
[1294, 1, 5, 246, 0, 4, 931, 12, 732] -> 3
[1, 5, 246, 0, 4, 931, 12, 732, 3] -> 508
[5, 246, 0, 4, 931, 12, 732, 3, 508] -> 131
[246,

In [13]:
%%time
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

torch.manual_seed(seed)

class ParagrafosDataset(Dataset):
  def __init__(self, paragrafos, vocab, context_size):
    # Salva o vocabulário
    self.vocab = vocab
    # Cria os inputs e target
    inputs = []
    targets = []

    for p in paragrafos:
      # O primeiro passo é pegar cada frase do parágrafo e encodar
      p_tokenizado = encode_sentence(p, self.vocab)
      # Só faz sentido considerar frases que tem no mínimo (context_size + 1) tokens
      if (len(p_tokenizado) <= context_size):
        continue

      # Agora vamos gerar os dados de treinamento para esse parágrafo
      p_inputs, p_targets = gera_inputs_e_targets_para_array(p_tokenizado, context_size)

      # Adiciona independentemente se tiver UKN ou não no input ou target
      inputs.extend(p_inputs)
      targets.extend(p_targets)

      # Apenas adiciona se o input ou o target não tiver nenhum UNK (código 0)
      #for p_um_input, p_um_target in zip(p_inputs, p_targets):
      #  if (0 not in p_um_input and p_um_target != 0):
      #    inputs.append(p_um_input)
      #    targets.append(p_um_target)

    # Mantém em cache
    self.inputs = torch.tensor(inputs)
    self.targets = torch.tensor(targets)

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, idx):
    return self.inputs[idx], self.targets[idx]

CPU times: user 1.88 s, sys: 400 ms, total: 2.28 s
Wall time: 5.14 s


In [14]:
teste_paragrafos = ["Depois, vendo que esta expedição não se realisava, e que seu braço e sua coragem de nada valião ao rei de Portugal"]
teste_dataset = ParagrafosDataset(teste_paragrafos, vocab, context_size)

print('Imprimindo o dataset')
for dados in teste_dataset:
  print(dados)

print('-------------------------')
print('Como deveria estar (testando se o dataset está considerando corretamente os parágrafos. Tem que descartar os que tem UNK (0)):')
for p in teste_paragrafos:
  # Faz o encode do parágrafo
  p_encodado = encode_sentence(p, vocab)
  inputs, targets = gera_inputs_e_targets_para_array(p_encodado, context_size)
  for inputs_targets in zip(inputs, targets):
    print(torch.tensor(inputs_targets[0]), torch.tensor(inputs_targets[1]))

Imprimindo o dataset
(tensor([ 63,   1, 275,   5, 120, 995,  13,   9,   0]), tensor(1))
(tensor([  1, 275,   5, 120, 995,  13,   9,   0,   1]), tensor(8))
(tensor([275,   5, 120, 995,  13,   9,   0,   1,   8]), tensor(5))
(tensor([  5, 120, 995,  13,   9,   0,   1,   8,   5]), tensor(20))
(tensor([120, 995,  13,   9,   0,   1,   8,   5,  20]), tensor(204))
(tensor([995,  13,   9,   0,   1,   8,   5,  20, 204]), tensor(8))
(tensor([ 13,   9,   0,   1,   8,   5,  20, 204,   8]), tensor(18))
(tensor([  9,   0,   1,   8,   5,  20, 204,   8,  18]), tensor(363))
(tensor([  0,   1,   8,   5,  20, 204,   8,  18, 363]), tensor(7))
(tensor([  1,   8,   5,  20, 204,   8,  18, 363,   7]), tensor(252))
(tensor([  8,   5,  20, 204,   8,  18, 363,   7, 252]), tensor(0))
(tensor([  5,  20, 204,   8,  18, 363,   7, 252,   0]), tensor(28))
(tensor([ 20, 204,   8,  18, 363,   7, 252,   0,  28]), tensor(550))
(tensor([204,   8,  18, 363,   7, 252,   0,  28, 550]), tensor(7))
(tensor([  8,  18, 363,   7, 2

Gera datasets de treinamento e de teste:

- Vou fazer a consideração de que a proporção é no total de parágrafos, e não no total do conjunto de dados. Como cada parágrafo tem um total de frases/palavras diferentes, o conjunto final não ficará com a proporção exatamente conforme esperado inicialmente. Entretanto, pensando que em um texto as coisas são mais ou menos distribuídas, espera-se que, no final, a proporção seja mais ou menos conforme a desejada.

- Depois de fazer isso, é necessário gerar novamente o vocabulário, mas considerando apenas o conjunto de treinamento.

In [15]:
from sklearn.model_selection import train_test_split

train_paragrafos, val_paragrafos = train_test_split(paragrafos, test_size=test_size, random_state=seed)

In [16]:
# Gera novamente o vocabulário, mas agora usando apenas os parágrafos de treinamento
vocab_size, vocab, most_frequent_words = gerar_vocabulario(train_paragrafos, vocab_size_desejado_sem_UNK)
print(vocab_size)

3001


In [17]:
# Gera os dataset de treino e validação
train_data = ParagrafosDataset(train_paragrafos, vocab, context_size)
val_data = ParagrafosDataset(val_paragrafos, vocab, context_size)

In [18]:
print(f'len(val_data): {len(val_data)}')
print(f'len(train_data): {len(train_data)}')
print(f'Proporção de teste: {len(val_data)/(len(train_data)+len(val_data))}')

len(val_data): 19009
len(train_data): 76020
Proporção de teste: 0.20003367393111576


# DataLoader

In [19]:
%%time
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

CPU times: user 265 µs, sys: 46 µs, total: 311 µs
Wall time: 318 µs


# Modelo

In [20]:
import torch.nn as nn

class LanguageModel(torch.nn.Module):
  def __init__(self, vocab_size, context_size, m, h):
    super(LanguageModel, self).__init__()

    self.C = nn.Embedding(vocab_size, m)
    self.d_plus_H = nn.Linear(in_features=context_size*m, out_features=h, bias=True)
    self.relu = nn.ReLU()
    self.b_plus_U = nn.Linear(in_features=h, out_features=vocab_size, bias=True)
    # Modelo do artigo:
    #self.U = nn.Linear(in_features=h, out_features=vocab_size, bias=False)
    #self.b_plus_W = nn.Linear(in_features=context_size*m, out_features=vocab_size, bias=True)

  def forward(self, w):
    # A fórmula é:
    # y = b + Wx + U*tanh(d + Hx)
    #
    # No exercício, o professor pediu para usar ReLU no lugar de tanh. Além disso,
    # comentou para usar duas camadas lineares. Então provavelmente estamos
    # fazendo é:
    # y = b + U*relu(d + Hx)
    # Que é similar ao original, mas considerando W = 0

    # x é uma entrada de tamanho context_size (no artigo é chamada de n)
    # O primeiro passo é manter os embeddings de x
    x = self.C(w)
    if x.dim() == 3: # Usando batchs
      batch_size, _, _ = x.shape
      x = x.view(batch_size, -1)
    elif x.dim() == 2: # Calculando sem usar batch, usando um tensor direto
      x = x.view(-1)
    # O segundo passo é fazer (d + Hx). Isso é uma transformação linear
    # A entrada é x (tamanho n*m) e a saída vai ser h (definida)
    o = self.d_plus_H(x)
    # O artigo calcula com tangente hiperbólica, mas o professor pediu com ReLU
    o = self.relu(o)
    # Passando pela segunda camada
    return self.b_plus_U(o)
    # return self.U(o) + self.b_plus_W(x) # Modelo do artigo

# Model instantiation
model = LanguageModel(vocab_size, context_size, m, h)

In [21]:
import numpy as np
# Testes com as dimensões
# Gera um embeddings
C = nn.Embedding(vocab_size, m)
# Considera que a entrada é um vetor de índice (tem que ser do tamanho de context_size) e calcula os embeddings
x = C(torch.tensor(np.random.randint(0, 10, size=context_size)))
print(x.shape)
# Achata o vetor
x = x.view(-1)
print(x.shape)
# Cria a primeira camada
d_plus_H = nn.Linear(in_features=context_size*m, out_features=h, bias=True)
o = d_plus_H(x)
print(o.shape)
# Passa por ReLu
o = nn.ReLU()(o)
print(o.shape)
# Última camada
b_plus_U = nn.Linear(in_features=h, out_features=vocab_size, bias=True)
o = b_plus_U(o)
print(o.shape)

torch.Size([9, 64])
torch.Size([576])
torch.Size([200])
torch.Size([200])
torch.Size([3001])


# Treinamento

In [22]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
  print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
  print('using CPU')

GPU: Tesla T4


In [23]:
import math
from tqdm import tqdm

def calcula_loss_e_perplexidade(model, loader):
  criterion = nn.CrossEntropyLoss(reduction='sum')
  with torch.no_grad(): # Garante que nenhum gradiente seja calculado
    model.eval()  # Coloca o modelo no modo de avaliação (não treinamento)
    loss = 0.0
    acc = 0
    for inputs, targets in tqdm(loader, desc='Calculando loss e perplexidade'):
      inputs = inputs.to(device)
      targets = targets.to(device)
      # Forward pass
      outputs = model(inputs)
      # Acumula a perda
      loss += criterion(outputs, targets)
      acc += len(targets)

    loss = loss/acc
    ppl = math.exp(loss)

    return loss, ppl

Cálcula a loss e a perplexidade antes do treinamento

In [24]:
def print_loss_ppl(msg, loss, ppl):
  print(f'{msg}. Loss: {loss:.2f}. Perplexidade: {ppl:.2f}\n')

In [25]:
# Model instantiation
model = LanguageModel(vocab_size, context_size, m, h)
model.to(device)

# Primeiro testa em um dataloader pequeno:
dataset_pequeno = ParagrafosDataset(paragrafos[0:15], vocab, context_size)
loader_pequeno = DataLoader(dataset_pequeno, batch_size=2, shuffle=False)

loss, ppl = calcula_loss_e_perplexidade(model, loader_pequeno)
print_loss_ppl('\nAntes de iniciar o treinamento', loss, ppl)

Calculando loss e perplexidade: 100%|██████████| 58/58 [00:01<00:00, 42.69it/s]


Antes de iniciar o treinamento. Loss: 8.05. Perplexidade: 3127.60



In [26]:
loss, ppl = calcula_loss_e_perplexidade(model, train_loader)
print_loss_ppl('\nAntes de iniciar o treinamento', loss, ppl)

Calculando loss e perplexidade: 100%|██████████| 594/594 [00:01<00:00, 333.86it/s]


Antes de iniciar o treinamento. Loss: 8.05. Perplexidade: 3142.88



In [27]:
import time
import torch.optim as optim

def treina_modelo(model, optimizer, train_loader, val_loader, num_epochs=num_epochs):
  print(f'------------------ ANTES DE INICIAR O TREINAMENTO ------------------')
  loss, ppl = calcula_loss_e_perplexidade(model, train_loader)
  print_loss_ppl(f'[TRAIN]', loss, ppl)

  loss, ppl = calcula_loss_e_perplexidade(model, val_loader)
  print_loss_ppl(f'[EVAL]', loss, ppl)


  criterion = nn.CrossEntropyLoss(reduction='mean')
  for epoch in range(num_epochs):
    model.train()
    start_time = time.time()  # Start time of the epoch
    print(f'------------------ [ÉPOCA {epoch+1}/{num_epochs}] ------------------')
    estimativa_loss_epoca_i = 0
    acc_dados = 0
    for inputs, targets in tqdm(train_loader, desc='Treinando modelo'):
      inputs = inputs.to(device)
      targets = targets.to(device)
      # Forward pass
      outputs = model(inputs)
      # Calcula loss no batch
      loss = criterion(outputs, targets)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      # Acumula a loss pra época atual
      # Obs.: isso é só uma estimativa para a loss na época i.
      # Como os pesos são atualizados após rodar cada batch, ao final da época
      # é esperado que a loss no conjunto de treinamento na verdade seja menor
      # do que o calculado dessa forma (o ajuste em cada batch tende a ir
      # convergindo e, consequentemente, diminuindo a loss)
      estimativa_loss_epoca_i += loss.item() * len(train_loader)
      acc_dados += len(train_loader)

    estimativa_loss_epoca_i = estimativa_loss_epoca_i / acc_dados
    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch

    print(f'Elapsed time: {epoch_duration:.2f} sec')

    loss, ppl = calcula_loss_e_perplexidade(model, train_loader)
    print_loss_ppl(f'[TRAIN]', loss, ppl)
    print_loss_ppl(f'[TRAIN ESTIMATIVA]', estimativa_loss_epoca_i, math.exp(estimativa_loss_epoca_i))

    loss, ppl = calcula_loss_e_perplexidade(model, val_loader)
    print_loss_ppl(f'[EVAL]', loss, ppl)

    checkpoint_path = f"modelo_epoca_{epoch+1}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_path)

In [28]:
def escrever_frase(modelo, vocab, most_frequent_words, entrada, context_size, n_proximas_palavras, descartar_ukn=True):
  if (n_proximas_palavras == 0):
    return entrada
  else:
    # Faz o encode da frase de entrada e considera apenas as últimas 'context_size'
    inputs = encode_sentence(entrada, vocab)
    # Pega só as context_size últimas
    inputs = inputs[len(inputs)-context_size:len(inputs)]

    with torch.no_grad():
      output = model(torch.tensor(inputs).to(device))
      softmax = nn.functional.softmax(output, dim=0)
      if descartar_ukn:
        valores, indices = softmax.topk(2, dim=0)
        melhor_not_ukn = indices[0].item() if indices[0].item() != 0 else indices[1].item()
        predicao = most_frequent_words[melhor_not_ukn]
      else:
        argmax = softmax.argmax(dim=0)
        predicao = most_frequent_words[argmax]

    # Substitui símbolos que foram trocados manualmente
    predicao = predicao.replace('SUBSTITUIRPORTRESPONTOS', '...')

  return escrever_frase(modelo, vocab, most_frequent_words, f'{entrada} {predicao}', context_size, n_proximas_palavras-1)

In [29]:
# Reinicializa o modelo
model = LanguageModel(vocab_size, context_size, m, h)
model.to(device)

# Escreve uma frase com o modelo sem estar treinado
frase = "O espectaculo que se ofereceu aos seus olhos causou"
print(escrever_frase(model, vocab, most_frequent_words, frase, context_size, 10))

# Treina o modelo
#optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.1)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
treina_modelo(model, optimizer, train_loader, val_loader, num_epochs=num_epochs)

O espectaculo que se ofereceu aos seus olhos causou succedeu plumas podesse dito filha labios travessa lutar cheia tremendo
------------------ ANTES DE INICIAR O TREINAMENTO ------------------


Calculando loss e perplexidade: 100%|██████████| 594/594 [00:01<00:00, 561.85it/s]


[TRAIN]. Loss: 7.99. Perplexidade: 2958.61



Calculando loss e perplexidade: 100%|██████████| 149/149 [00:00<00:00, 511.61it/s]


[EVAL]. Loss: 7.99. Perplexidade: 2951.05

------------------ [ÉPOCA 1/50] ------------------


...

Elapsed time: 1.29 sec


Calculando loss e perplexidade: 100%|██████████| 594/594 [00:00<00:00, 876.19it/s]


[TRAIN]. Loss: 0.55. Perplexidade: 1.74

[TRAIN ESTIMATIVA]. Loss: 0.72. Perplexidade: 2.05



Calculando loss e perplexidade: 100%|██████████| 149/149 [00:00<00:00, 832.90it/s]


[EVAL]. Loss: 14.12. Perplexidade: 1358629.93



In [30]:
def recupera_modelo(model, epoca):
  # Recupera o modelo salvo na época x
  checkpoint_path = f"modelo_epoca_{epoca}.pth"
  # Carregar o estado do checkpoint
  checkpoint = torch.load(checkpoint_path)
  # Aplicar o estado do modelo e otimizador carregados
  model.load_state_dict(checkpoint['model_state_dict'])

O conjunto foi treinado com 50 épocas numa tentativa de fazer um overfit do modelo e verificar se ele consegue reproduzir mais ou menos o conjunto de treinamento:

In [31]:
# Overfit no modelo pra ver se ele consegue decorar as frases do conjunto de treinamento
def completa_frase_do_conjunto(context_size, paragrafos, indices, idx_modelo_overfit):
  for i in indices:
    frase_esperada = paragrafos[i]
    palavras_na_frase = frase_esperada.split(' ')
    palavras_na_frase = palavras_na_frase[0:context_size]

    if len(palavras_na_frase) == context_size:
      frase = ' '.join(palavras_na_frase)
      recupera_modelo(model, idx_modelo_overfit)
      print('-----------------------------------------------------------------')
      print(f'Testando para o índice {i}')
      print(f'Modelo da epoca {idx_modelo_overfit}:')
      print('Início:  ', frase)
      print('Correta: ', frase_esperada)
      print('Gerada:  ', escrever_frase(model, vocab, most_frequent_words, frase, context_size, 30, descartar_ukn=False))

completa_frase_do_conjunto(context_size, train_paragrafos, [0, 1, 2, 3, 55, 61, 76, 78, 388, 555, 1000], num_epochs)

-----------------------------------------------------------------
Testando para o índice 0
Modelo da epoca 50:
Início:   Tambem elle viu a luz das janellas se reflectir
Correta:  Tambem elle viu a luz das janellas se reflectir de fronte; e esperou que a noite se adiantasse, e toda a casa dormisse.
Gerada:   Tambem elle viu a luz das janellas se reflectir de fronte ; e esperou que o tinha entrar , tão de mais para escura - se do mais do que o puro a morte da força acabava que a
-----------------------------------------------------------------
Testando para o índice 1
Modelo da epoca 50:
Início:   --É de Caparica, mas do bom. Deste cá não
Correta:  --É de Caparica, mas do bom. Deste cá não vem!
Gerada:   --É de Caparica, mas do bom. Deste cá não vem ! elle alvaro desgraça - os de . , cecilia - se ella ! para o alpendre . é - la selvagem , se se o fructo ; tu
-----------------------------------------------------------------
Testando para o índice 2
Modelo da epoca 50:
Início:   --Dava-lhe

Testa com uma fase qualquer, mas considerando todos os modelos gerados nas primeiras 10 épocas (só pra ver o que ele está gerando):

In [32]:
frase = "Se se tratasse de sua vida, Pery teria sangue"
print(frase)
for epoca in range(1, min(num_epochs+1, 11)):
  recupera_modelo(model, epoca)
  #print(f'Modelo da epoca {epoca}:', escrever_frase(model, vocab, most_frequent_words, frase, context_size, 30, descartar_ukn=False))
  print(f'Modelo da epoca {epoca}:', escrever_frase(model, vocab, most_frequent_words, frase, context_size, 30, descartar_ukn=True))

Se se tratasse de sua vida, Pery teria sangue
Modelo da epoca 1: Se se tratasse de sua vida, Pery teria sangue , e que o seu . não se o seu de um , e de um . de sua senhora . de sua senhora , e que o seu que
Modelo da epoca 2: Se se tratasse de sua vida, Pery teria sangue , e a sua vida , e o fidalgo , e o seu espirito , e o seu , e que o tinha , e a sua vida , e
Modelo da epoca 3: Se se tratasse de sua vida, Pery teria sangue , e de sua senhora , e - se um olhar de uma de que se passava ; a sua senhora de uma grande de uma grande de uma que
Modelo da epoca 4: Se se tratasse de sua vida, Pery teria sangue , e de sua familia . de sua familia . da vida de d . antonio de mariz , disse - se , e de sua familia , e a
Modelo da epoca 5: Se se tratasse de sua vida, Pery teria sangue , que se havião a sua vida , o indio ; o meu de sua prima , que elle - se de seu pai , e a - lhe a
Modelo da epoca 6: Se se tratasse de sua vida, Pery teria sangue , tinha passado , e o que se tinha passado ; sabia

Continua alguns parágrafos da base de avaliação usando o modelo treinado na época que deu menor perplexidade no conjunto de treino.

In [39]:
epoca_do_modelo = 3
completa_frase_do_conjunto(context_size, val_paragrafos, [1, 2, 4, 5, 8, 9, 18, 20, 21, 30], epoca_do_modelo)

-----------------------------------------------------------------
Testando para o índice 1
Modelo da epoca 3:
Início:   O que soffreu quando Cecilia no seu desespero pela
Correta:  O que soffreu quando Cecilia no seu desespero pela morte de seu pai o accusava por tê-la salvado, e lhe dava ordem de leva-la ao lugar onde repousavão as cinzas do velho fidalgo, é impossivel de descrever.
Gerada:   O que soffreu quando Cecilia no seu desespero pela <unk> . » de mariz , que se tinha de um gesto de sua senhora . e de um homem que de janeiro de uma que havia lhe tinha de
-----------------------------------------------------------------
Testando para o índice 4
Modelo da epoca 3:
Início:   --Ah! nunca! Não me peças uma cousa impossivel, Cecilia!
Correta:  --Ah! nunca! Não me peças uma cousa impossivel, Cecilia! Já sabes de mais; não me obrigues a morrer a teus pés de vergonha.
Gerada:   --Ah! nunca! Não me peças uma cousa impossivel, Cecilia! ... a moça . antonio de mariz , e não podia o que ha

Calcula o total de parâmetros da rede

In [41]:
# Total de parâmetros teórico:
total_embeddings = vocab_size * m
total_camada_1 = context_size * m * h + h # elementos da matriz + bias
total_camada_2 = h * vocab_size + vocab_size # elementos da matriz + bias

print(f'Total embeddings: {total_embeddings}')
print(f'Total camada 1: {total_camada_1}')
print(f'Total camada 2: {total_camada_2}')
print(total_embeddings + total_camada_1 + total_camada_2, '<- somando tudo')

# Total de parâmetros extraído do modelo:
print(sum(p.numel() for p in model.parameters()), '<- sum(p.numel() for p in model.parameters())')

Total embeddings: 192064
Total camada 1: 115400
Total camada 2: 603201
910665 <- somando tudo
910665 <- sum(p.numel() for p in model.parameters())
